# Fine-Tuning do Modelo LLama3.1

[inserir descrição sobre estarmos fazendo fine tunning de modelo com X parametros com Y dados]

SyntaxError: invalid syntax (<ipython-input-27-92cbf9cbbfb4>, line 1)

## Importando o Unsloath

In [4]:
%%capture
!pip install unsloth

# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install triton
!pip install bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # comprimento máximo de tokens que o modelo será capaz de processar
dtype = None # none para autodetecção
load_in_4bit = True # Habilita a quantização de 4 bits. Isso reduz o consumo de memória do modelo, permitindo que ele seja executado em hardware com menos memória, sem causar erros de falta de memória

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = "unsloth/Meta-Llama-3.1-8B",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post1: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


**LoRA (Low-Rank Adaptation)**

Basicamente é uma técnica de fine-tunning de LLMs, que visa diminuir a quantidade de parâmetros treináveis durante o fine tunning sem comprometer a precisão e a qualidade do modelo.

Os grandes modelos de linguagem, como GPT-3, LLaMA e outros, possuem bilhões de parâmetros, o que torna o ajuste fino completo (atualizar todos os parâmetros durante o treinamento) um processo extremamente intensivo em termos de memória e tempo. O ajuste fino convencional consome muito recurso computacional e é inviável em dispositivos com pouca capacidade, como GPUs com menor memória.

*Funcionamento:*
Em vez de ajustar todos os parâmetros do modelo, o LoRA foca em projetar atualizações apenas em determinadas projeções (geralmente nos mecanismos de atenção) usando decomposição matricial de baixa ordem (low-rank). A

In [3]:
model = FastLanguageModel.get_peft_model(
  model,
  r = 16, # quanto maior, mais parâmetros serão treinados


  # Especifica quais projeções dentro do modelo serão adaptadas com LoRA.
  # Projeções como q_proj, k_proj, v_proj, e o_proj referem-se a partes das
  # camadas de autoatenção (query, key, value e output projections) usadas
  # em mecanismos de atenção, como o de transformadores.

  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],

  lora_alpha = 16, # influência do lora no aprendizado
  lora_dropout = 0,
  bias = "none",

  use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
  random_state = 3407,
  use_rslora = False,
  loftq_config = None,
)

Unsloth 2024.9.post1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Preparação dos dados para treino

In [4]:
from datasets import Dataset
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:


# Carregar o arquivo CSV usando pandas
csv_file_path = "/content/drive/Shareddrives/grupo3moshi/chats_clients.csv"
data = pd.read_csv(csv_file_path)

# Verificar as primeiras linhas para garantir que o arquivo foi carregado corretamente
print(data.head())


# Definir o EOS_TOKEN usando o tokenizer
EOS_TOKEN = tokenizer.eos_token

# Função para converter o dataframe em um formato apropriado para o treino
def prepare_data(data):
    inputs = []
    outputs = []

    for _, row in data.iterrows():
        user_input = row['Pergunta']   # Usar a coluna 'Pergunta' como Input
        assistant_response = row['Intencao']  # Usar a coluna 'Intencao' como Response

        # Verificar se os campos não estão vazios
        if pd.notnull(user_input) and pd.notnull(assistant_response):
            inputs.append(user_input)
            outputs.append(assistant_response)

    return {
        "input": inputs,
        "output": outputs,
    }

# Converter os dados
dataset_dict = prepare_data(data)

# Criar dataset usando a biblioteca datasets
dataset = Dataset.from_dict(dataset_dict)


   No                              Intencao  \
0   1                        Como depositar   
1   2                    Como fazer remessa   
2   3                      Tempo de remessa   
3   4  Pedido de envio via metodo "ByPhone"   
4   5  Pedido de envio via metodo "ByPhone"   

                                            Pergunta  \
0  Boa dia.tudo bem?eu gostaria de saber sobre aq...   
1                     Como enviar dinheiro do Japão?   
2  Quanto tempo levará para o beneficiário recebe...   
3  Boa tarde\nAcabei de fazer a transferência de ...   
4  Poderia fazer a remessa de 22yenes para o BBB ...   

                                            Resposta  
0  Bom dia! Sim, o sr pode utilizar o cartão de d...  
1  Para se inscrever no serviço de remessa, por f...  
2  Via de regra, as remessas serão pagas via PIX ...  
3  iremos processar a sua solicitacao.\nMuito obr...  
4  iremos processar a sua solicitacao.\nMuito obr...  


In [6]:
intent = data['Intencao'].unique()
print(intent)

['Como depositar' 'Como fazer remessa' 'Tempo de remessa'
 'Pedido de envio via metodo "ByPhone"' 'Como se inscrever'
 'Confirmacao de cambio/taxas' 'Envio via Deposit Code'
 'Registro/Atualizacao de Documento' 'Solicitacao de cartao de remessas'
 'Cadastro de beneficiario' 'Acesso a conta'
 'Termos e condicoes do servico' 'Problemas de remessa'
 'Tempo de entrega do cartao' 'Reembolso' 'Regras do servico'
 'Atualizacao de dados cadastrais' 'Cancelamento']


In [7]:
alpaca_prompt = """

  Você é um classificador de intenções de perguntas em um chat de atendimento da empresa Brastel.

  ## Regras
  - Nao sair dos limites de categoria possiveis
  - Classificar apenas uma pergunta

  ## Categorias Possiveis:
  'Pedido de envio via metodo "ByPhone"', 'Como se inscrever', 'Confirmacao de cambio/taxas', 'Envio via Deposit Code', 'Registro/Atualizacao de Documento',
  'Solicitacao de cartao de remessas', 'Cadastro de beneficiario', 'Acesso a conta',
 'Termos e condicoes do servico', 'Problemas de remessa',
 'Tempo de entrega do cartao', 'Reembolso' 'Regras do servico',
 'Atualizacao de dados cadastrais', 'Cancelamento',
 'Problemas/Duvidas sobre deposito',
 'Problemas/Duvidas de atualizacao de dados cadastrais',
 'Problemas/Duvidas sobre remessas', 'Dificuldades com utilizacao do App',
 'Como depositar ', 'Tempo para depositos refletirem no saldo',
 'Confirmacao de saldo'


De acordo com a pergunta abaixo, qual das intenções acima se encaixa? Responda apenas com a intenção.
### Pergunta:
{}

### Resposta:
{}"""

def formatting_prompts_func(examples):
    texts = []
    for input_text, output in zip(examples["input"], examples["output"]):
        text = alpaca_prompt.format(input_text, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}


# Aplicar a função de formatação
dataset = dataset.map(formatting_prompts_func, batched=True)

# Verificar as primeiras linhas do dataset formatado
print(dataset[10])

Map:   0%|          | 0/505 [00:00<?, ? examples/s]

{'input': 'Como posso me inscrever no serviço de remessa?', 'output': 'Como se inscrever', 'text': '\n\n  Você é um classificador de intenções de perguntas em um chat de atendimento da empresa Brastel.\n\n  ## Regras\n  - Nao sair dos limites de categoria possiveis\n  - Classificar apenas uma pergunta\n\n  ## Categorias Possiveis:\n  \'Pedido de envio via metodo "ByPhone"\', \'Como se inscrever\', \'Confirmacao de cambio/taxas\', \'Envio via Deposit Code\', \'Registro/Atualizacao de Documento\', \n  \'Solicitacao de cartao de remessas\', \'Cadastro de beneficiario\', \'Acesso a conta\',\n \'Termos e condicoes do servico\', \'Problemas de remessa\',\n \'Tempo de entrega do cartao\', \'Reembolso\' \'Regras do servico\',\n \'Atualizacao de dados cadastrais\', \'Cancelamento\',\n \'Problemas/Duvidas sobre deposito\',\n \'Problemas/Duvidas de atualizacao de dados cadastrais\',\n \'Problemas/Duvidas sobre remessas\', \'Dificuldades com utilizacao do App\',\n \'Como depositar \', \'Tempo para

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 600000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/505 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


**Inferindo**

In [9]:

# Enable native 2x faster inference with FastLanguageModel
FastLanguageModel.for_inference(model)

# Format the input for the model, providing instruction, input, and leaving the output blank
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Como faço para enviar dinheiro para o Brasil?",  # input
            ""  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt"  # Return PyTorch tensors
).to("cuda")  # Move tensors to GPU if available

# Generate a response from the model
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decode the output tokens to text
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the decoded response
print(decoded_outputs)


['\n\n  Você é um classificador de intenções de perguntas em um chat de atendimento da empresa Brastel.\n\n  ## Regras\n  - Nao sair dos limites de categoria possiveis\n  - Classificar apenas uma pergunta\n\n  ## Categorias Possiveis:\n  \'Pedido de envio via metodo "ByPhone"\', \'Como se inscrever\', \'Confirmacao de cambio/taxas\', \'Envio via Deposit Code\', \'Registro/Atualizacao de Documento\', \n  \'Solicitacao de cartao de remessas\', \'Cadastro de beneficiario\', \'Acesso a conta\',\n \'Termos e condicoes do servico\', \'Problemas de remessa\',\n \'Tempo de entrega do cartao\', \'Reembolso\' \'Regras do servico\',\n \'Atualizacao de dados cadastrais\', \'Cancelamento\',\n \'Problemas/Duvidas sobre deposito\',\n \'Problemas/Duvidas de atualizacao de dados cadastrais\',\n \'Problemas/Duvidas sobre remessas\', \'Dificuldades com utilizacao do App\',\n \'Como depositar \', \'Tempo para depositos refletirem no saldo\',\n \'Confirmacao de saldo\'\n\n\nDe acordo com a pergunta abaixo,

In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "é possivel cancelar minha remessa?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

<|begin_of_text|>

  Você é um classificador de intenções de perguntas em um chat de atendimento da empresa Brastel.

  ## Regras
  - Nao sair dos limites de categoria possiveis
  - Classificar apenas uma pergunta

  ## Categorias Possiveis:
  'Pedido de envio via metodo "ByPhone"', 'Como se inscrever', 'Confirmacao de cambio/taxas', 'Envio via Deposit Code', 'Registro/Atualizacao de Documento', 
  'Solicitacao de cartao de remessas', 'Cadastro de beneficiario', 'Acesso a conta',
 'Termos e condicoes do servico', 'Problemas de remessa',
 'Tempo de entrega do cartao', 'Reembolso' 'Regras do servico',
 'Atualizacao de dados cadastrais', 'Cancelamento',
 'Problemas/Duvidas sobre deposito',
 'Problemas/Duvidas de atualizacao de dados cadastrais',
 'Problemas/Duvidas sobre remessas', 'Dificuldades com utilizacao do App',
 'Como depositar ', 'Tempo para depositos refletirem no saldo',
 'Confirmacao de saldo'


De acordo com a pergunta abaixo, qual das intenções acima se encaixa? Responda ape

In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Fiz um deposito ontem quando chega na minha conta?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

<|begin_of_text|>

  Você é um classificador de intenções de perguntas em um chat de atendimento da empresa Brastel.

  ## Regras
  - Nao sair dos limites de categoria possiveis
  - Classificar apenas uma pergunta

  ## Categorias Possiveis:
  'Pedido de envio via metodo "ByPhone"', 'Como se inscrever', 'Confirmacao de cambio/taxas', 'Envio via Deposit Code', 'Registro/Atualizacao de Documento', 
  'Solicitacao de cartao de remessas', 'Cadastro de beneficiario', 'Acesso a conta',
 'Termos e condicoes do servico', 'Problemas de remessa',
 'Tempo de entrega do cartao', 'Reembolso' 'Regras do servico',
 'Atualizacao de dados cadastrais', 'Cancelamento',
 'Problemas/Duvidas sobre deposito',
 'Problemas/Duvidas de atualizacao de dados cadastrais',
 'Problemas/Duvidas sobre remessas', 'Dificuldades com utilizacao do App',
 'Como depositar ', 'Tempo para depositos refletirem no saldo',
 'Confirmacao de saldo'


De acordo com a pergunta abaixo, qual das intenções acima se encaixa? Responda ape